In [1]:
import samtools
# working_dir = 'C:/Users/Samuel_DeLap.UMLADCO/Documents/DeLap/AxoChewing'
working_dir = 'C:/Users/sjcde/OneDrive/Documents/AxoChewing/'

Loading DLC 2.2.3...


c:\Users\sjcde\.conda\envs\DEEPLABCUT\lib\site-packages\deeplabcut\__init__.py:81: UserWarning: 
        As PyTorch is not installed, unsupervised identity learning will not be available.
        Please run `pip install torch`, or ignore this warning.
        
  warnings.warn(


In [2]:
samtools.autocorrect(working_dir, threshold=5)

1600.0


UnboundLocalError: local variable 'subimage_float' referenced before assignment

In [3]:
# Vars and imports
import cv2
import math
import pandas as pd
import numpy as np
frame = cv2.imread('sample_frame.jpg')
new_data_path = working_dir +'/trials'
trial = '22JUL2021_Axo18_t04_c_strikeNchews'
iteration = 0
search_area = 15
cam = 'cam1'
hdf = pd.read_hdf(new_data_path + '/' + trial + '/' + 'it' + str(iteration) + '/' + trial + '-Predicted2DPoints.h5')

In [6]:
# Image processing
frame = samtools.filter_image(frame, krad=10)
# For each marker in the frame
parts_unique = samtools.getBodypartsFromXmaExport(working_dir)
# DEBUG
for part in parts_unique:
    # Find point and offsets
    x_float = hdf.loc[0, part + '_' + cam + '_X']
    y_float = hdf.loc[0, part + '_' + cam + '_Y']
    x_start = int(x_float-search_area+0.5)
    y_start = int(y_float-search_area+0.5)
    x_end = int(x_float+search_area+0.5)
    y_end = int(y_float+search_area+0.5)

    # Preprocessing
    # Crop image to marker vicinity
    subimage = frame[y_start:y_end, x_start:x_end]
    # Convert To float
    subimage_float = subimage.astype(np.float32)
    # Blurred image
    radius = int(1.5 * 2 + 0.5) #5 might be too high
    sigma = radius * math.sqrt(2 * math.log(255)) - 1
    subimage_blurred = cv2.GaussianBlur(subimage_float, (2 * radius + 1, radius + 1), sigma)
    # Subtract Background
    subimage_diff = subimage_float-subimage_blurred
    subimage_diff = cv2.normalize(subimage_diff, None, 0,255,cv2.NORM_MINMAX).astype(np.uint8)
    # Median
    subimage_median = cv2.medianBlur(subimage_diff, 3)
    # LUT
    subimage_median = samtools.filter_image(subimage_median, krad=3)

    # Convert to grayscale
    gray = cv2.cvtColor(subimage_float, cv2.COLOR_BGR2GRAY)

    # Edge detection
    edges = cv2.Canny(gray, threshold1=30, threshold2=100)

    # Gaussian blur
    gaussian = cv2.GaussianBlur(edges, (3,3), 1.3)

    # Thresholding
    ret, subimage_threshold =  cv2.threshold(gaussian, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    # Find contours
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE, offset=(x_start,y_start))
    # DEBUG contours
    # print("Detected "+str(len(contours))+" contours in "+str(search_area)+"*"+str(search_area)+" neighborhood of marker "+part+' in Camera '+cam[-1])
    contours_im = [contour-[x_start, y_start] for contour in contours]

    # Find closest contour
    dist = 1000
    best_index = -1
    detected_centers = {}
    for i in range(len(contours)):
        detected_center, circle_radius = cv2.minEnclosingCircle(contours[i])
        distTmp = math.sqrt((x_float - detected_center[0])**2 + (y_float - detected_center[1])**2)
        detected_centers[round(distTmp, 4)] = detected_center
        if distTmp < dist:
            best_index = i
            dist = distTmp
        if best_index >= 0:
            detected_center, _ = cv2.minEnclosingCircle(contours[best_index])
            detected_center_im, _ = cv2.minEnclosingCircle(contours_im[best_index])
    samtools.show_crop(subimage, center=search_area, contours = [contours_im[best_index]], detected_marker = detected_center_im)
    samtools.show_crop(gray, center=search_area, contours=contours_im,detected_marker = detected_center_im)
    samtools.show_crop(edges, center=search_area, contours=contours_im,detected_marker = detected_center_im)
    samtools.show_crop(gaussian, center=search_area, contours = [contours_im[best_index]], detected_marker = detected_center_im)
    samtools.show_crop(subimage_threshold, center=search_area, contours = [contours_im[best_index]], detected_marker = detected_center_im)

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\smooth.dispatch.cpp:294: error: (-215:Assertion failed) ksize.width > 0 && ksize.width % 2 == 1 && ksize.height > 0 && ksize.height % 2 == 1 in function 'cv::createGaussianKernels'
